In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from numpy.random import multivariate_normal, permutation
import pandas as pd
from pandas import DataFrame, Series
from tensorflow.examples.tutorials.mnist import input_data

np.random.seed(20160704)
tf.set_random_seed(20160704)


In [2]:
def unpickle(file):
    import pickle
    with open(file, 'rb') as fo:
        dict = pickle.load(fo, encoding='bytes')
    return dict


In [ ]:
sess = tf.InteractiveSession()


#トレーニングデータ準備
train_file_1 = unpickle("data_batch_1")
train_file_2 = unpickle("data_batch_2")
train_file_3 = unpickle("data_batch_3")
train_file_4 = unpickle("data_batch_4")
train_file_5 = unpickle("data_batch_5")

data_1 = np.array(train_file_1[b'data'])
data_2 = np.array(train_file_2[b'data'])
data_3 = np.array(train_file_3[b'data'])
data_4 = np.array(train_file_4[b'data'])
data_5 = np.array(train_file_5[b'data'])


labels_1 = np.array(train_file_1[b'labels'])
labels_2 = np.array(train_file_2[b'labels'])
labels_3 = np.array(train_file_3[b'labels'])
labels_4 = np.array(train_file_4[b'labels'])
labels_5 = np.array(train_file_5[b'labels'])


data = np.concatenate((data_1,data_2,data_3,data_4,data_5), axis=0)
labels = np.concatenate((labels_1,labels_2, labels_3,labels_4,labels_5) ,axis=0)
data = data.reshape([-1,3,32,32]).transpose([0,2,3,1])
data = data.reshape([-1,3*32*32])

#トレーニング、ラベルをonehot にする
#labels_oh = np.zeros([len(labels),10])
labels_oh=tf.one_hot(labels.flatten(),10).eval()
#labels_oh[np.arange(len(labels)),labels]  =1

#テストデータ準備
test_file = unpickle("test_batch")
test_labels = np.array(test_file[b'labels'])
test_data = np.array(test_file[b'data'])

#テスト、ラベルをonehot にする
test_labels_oh = np.zeros([len(test_labels),10])
test_labels_oh=tf.one_hot(test_labels.flatten(),10).eval()
#test_labels_oh[np.arange(len(test_labels)),test_labels]  =1


x = tf.placeholder(tf.float32, [None,3072])
x_images = tf.reshape(x,[-1,32,32,3])

num_filter1 = 32
w1_conv = tf.Variable(tf.truncated_normal([5,5,3,num_filter1], stddev = 0.1))
b1_conv = tf.Variable(tf.constant(0.1, shape=[num_filter1]))
h1_conv = tf.nn.conv2d(x_images, w1_conv, strides=[1,1,1,1], padding="SAME")
h1_conv_cutoff = tf.nn.relu(h1_conv+b1_conv)
h1_pool = tf.nn.max_pool(h1_conv_cutoff, ksize = [1,2,2,1], strides = [1,2,2,1], padding = "SAME")

num_filter2 = 64
w2_conv = tf.Variable(tf.truncated_normal([5,5,num_filter1 ,num_filter2], stddev = 0.1))
b2_conv = tf.Variable(tf.constant(0.1, shape=[num_filter2]))
h2_conv = tf.nn.conv2d(h1_pool, w2_conv, strides=[1,1,1,1], padding="SAME")
h2_conv_cutoff = tf.nn.relu(h2_conv+b2_conv)
h2_pool = tf.nn.max_pool(h2_conv_cutoff, ksize = [1,2,2,1], strides = [1,2,2,1], padding = "SAME")
h2_pool_flat = tf.reshape(h2_pool, [-1, 8*8*num_filter2])

num_units1 = 8*8*num_filter2
num_units2 = 1024

w2 =  tf.Variable(tf.truncated_normal([num_units1 ,num_units2], stddev = 0.1))
b2 = tf.Variable(tf.constant(0.1, shape=[num_units2]))
hidden2 = tf.nn.relu(tf.matmul(h2_pool_flat,w2) + b2)

keep_prob = tf.placeholder(tf.float32)
hidden_drop =  tf.nn.dropout(hidden2, keep_prob)
w0 = tf.Variable(tf.truncated_normal([num_units2, 10],stddev = 0.1))
b0 = tf.Variable(tf.constant(0.1, shape=[10]))
p = tf.nn.softmax(tf.matmul(hidden_drop,w0) + b0)



t = tf.placeholder(tf.float32, [None, 10])


logits=tf.matmul(hidden_drop,w0) + b0
entropy = tf.nn.softmax_cross_entropy_with_logits(labels=t, logits=logits)
loss = tf.reduce_mean(entropy, name='loss')


correct_prediction = tf.equal(tf.argmax(p,1), tf.argmax(t,1))
train = tf.train.AdamOptimizer(0.0001).minimize(loss)
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

sess.run(tf.global_variables_initializer())


In [ ]:

i = 0
for _ in range(10000):
    i += 1
    batch_s = 50
    sess.run(train, feed_dict={x: data[ batch_s*i: batch_s * (i+1),:], t: labels_oh[ batch_s* i :  batch_s * (i+1),:], keep_prob:0.5})   
    if i%100 == 0:
        loss_val, acc_val = [],[]
        loss_val, acc_val = sess.run([loss, accuracy], feed_dict={x:test_data, t:test_labels_oh, keep_prob:1.0})        
        print ('Step: %d, Loss: %f, Accuracy: %f'% (i, loss_val, acc_val))


Step: 100, Loss: 382.212036, Accuracy: 0.129000
Step: 200, Loss: 240.629547, Accuracy: 0.126000
Step: 300, Loss: 156.172470, Accuracy: 0.124600
Step: 400, Loss: 114.790047, Accuracy: 0.114700
Step: 500, Loss: 80.984573, Accuracy: 0.120200
Step: 600, Loss: 47.751659, Accuracy: 0.122400
Step: 700, Loss: 22.306808, Accuracy: 0.115600
Step: 800, Loss: 10.511800, Accuracy: 0.110600
Step: 900, Loss: 6.411598, Accuracy: 0.106000
Step: 1000, Loss: 4.584335, Accuracy: 0.103900
Step: 1100, Loss: 4.552110, Accuracy: 0.103800
Step: 1200, Loss: 4.552110, Accuracy: 0.103800
Step: 1300, Loss: 4.552110, Accuracy: 0.103800
Step: 1400, Loss: 4.552110, Accuracy: 0.103800
Step: 1500, Loss: 4.552110, Accuracy: 0.103800
Step: 1600, Loss: 4.552110, Accuracy: 0.103800
Step: 1700, Loss: 4.552110, Accuracy: 0.103800
Step: 1800, Loss: 4.552110, Accuracy: 0.103800
Step: 1900, Loss: 4.552110, Accuracy: 0.103800
Step: 2000, Loss: 4.552110, Accuracy: 0.103800
Step: 2100, Loss: 4.552110, Accuracy: 0.103800
Step: 2200

1
2
3
4
5
6
7
8
9
